In [1]:
import numpy as np
import time
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import gzip

df = pd.read_table("SNV/MANIFEST.txt")
fileslist = []
header = []
for file_name in df[0:33]["filename"]:
    fileslist.append(file_name)
with gzip.open("SNV/"+fileslist[0],"rb") as f:
    print(f)
    i = 0
    while i <=4:
        header.append(f.readline())
        i += 1
    mf = pd.read_csv(f, delimiter = "\t")
    table = mf[0:]

<gzip _io.BufferedReader name='SNV/02747363-f04a-4ba6-a079-fe4f87853788/TCGA.UCS.mutect.02747363-f04a-4ba6-a079-fe4f87853788.DR-10.0.somatic.maf.gz' 0x26864a757c0>


In [7]:
mf

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,...,FILTER,CONTEXT,src_vcf_id,tumor_bam_uuid,normal_bam_uuid,case_id,GDC_FILTER,COSMIC,MC3_Overlap,GDC_Validation_Status
0,SLC6A9,6536,BI,GRCh38,chr1,44008472,44008472,+,Silent,SNP,...,PASS,ACACCGGCGCA,85b191fa-e8f6-4d97-bc13-0bc236cee6c8,3d52eda9-5f36-4a55-8a3c-07a576a0ef1f,6c3037f4-9838-4b36-9551-99e91cb85ef0,14213209-2217-4812-9a19-d9b2b6718467,NaN,COSM6063461;COSM6063462,True,Valid
1,PLK3,1263,BI,GRCh38,chr1,44803280,44803280,+,Missense_Mutation,SNP,...,PASS,CCCCCGATCGA,85b191fa-e8f6-4d97-bc13-0bc236cee6c8,3d52eda9-5f36-4a55-8a3c-07a576a0ef1f,6c3037f4-9838-4b36-9551-99e91cb85ef0,14213209-2217-4812-9a19-d9b2b6718467,NaN,NaN,True,Valid
2,C1orf168,199920,BI,GRCh38,chr1,56740711,56740711,+,Silent,SNP,...,PASS,TCTTTCGACTT,85b191fa-e8f6-4d97-bc13-0bc236cee6c8,3d52eda9-5f36-4a55-8a3c-07a576a0ef1f,6c3037f4-9838-4b36-9551-99e91cb85ef0,14213209-2217-4812-9a19-d9b2b6718467,common_in_exac,NaN,True,Unknown
3,GBA,2629,BI,GRCh38,chr1,155233116,155233116,+,3'Flank,SNP,...,PASS,GCATGCAGCGG,85b191fa-e8f6-4d97-bc13-0bc236cee6c8,3d52eda9-5f36-4a55-8a3c-07a576a0ef1f,6c3037f4-9838-4b36-9551-99e91cb85ef0,14213209-2217-4812-9a19-d9b2b6718467,NaN,NaN,False,Unknown
4,FCRL4,83417,BI,GRCh38,chr1,157581530,157581530,+,Splice_Site,SNP,...,PASS,ACAAACCTGAC,85b191fa-e8f6-4d97-bc13-0bc236cee6c8,3d52eda9-5f36-4a55-8a3c-07a576a0ef1f,6c3037f4-9838-4b36-9551-99e91cb85ef0,14213209-2217-4812-9a19-d9b2b6718467,NonExonic,NaN,True,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10444,RS1,6247,BI,GRCh38,chrX,18642139,18642139,+,Silent,SNP,...,PASS,CGGTCCGAGTT,1d4f6015-1138-4d11-ae68-5034b9af7511,810c44b5-8901-4877-8ca6-bedfd040e0c2,e052d1b5-8959-43df-916a-8dce7977fdc0,200f77a5-803c-4891-980b-5c3b08c91254,NaN,NaN,True,Unknown
10445,UBA1,7317,BI,GRCh38,chrX,47201524,47201524,+,Missense_Mutation,SNP,...,PASS,CGGGTTTGAGA,1d4f6015-1138-4d11-ae68-5034b9af7511,810c44b5-8901-4877-8ca6-bedfd040e0c2,e052d1b5-8959-43df-916a-8dce7977fdc0,200f77a5-803c-4891-980b-5c3b08c91254,NaN,NaN,True,Unknown
10446,PGAM4,441531,BI,GRCh38,chrX,77969434,77969434,+,Missense_Mutation,SNP,...,PASS,CACTGTCCAGA,1d4f6015-1138-4d11-ae68-5034b9af7511,810c44b5-8901-4877-8ca6-bedfd040e0c2,e052d1b5-8959-43df-916a-8dce7977fdc0,200f77a5-803c-4891-980b-5c3b08c91254,NaN,NaN,True,Unknown
10447,GPRASP1,9737,BI,GRCh38,chrX,102657224,102657224,+,Missense_Mutation,SNP,...,PASS,TCAGCCTGATC,1d4f6015-1138-4d11-ae68-5034b9af7511,810c44b5-8901-4877-8ca6-bedfd040e0c2,e052d1b5-8959-43df-916a-8dce7977fdc0,200f77a5-803c-4891-980b-5c3b08c91254,NaN,NaN,True,Unknown


In [9]:
Hugo_Symbol = table.loc[:,["Hugo_Symbol"]]
Y = table.loc[:,["Entrez_Gene_Id"]] #independent variables
print(Y)
print(Hugo_Symbol)
X = table.drop(["Hugo_Symbol","Entrez_Gene_Id"], axis = 1) #dependent variables
X

       Entrez_Gene_Id
0                6536
1                1263
2              199920
3                2629
4               83417
...               ...
10444            6247
10445            7317
10446          441531
10447            9737
10448          114824

[10449 rows x 1 columns]
      Hugo_Symbol
0          SLC6A9
1            PLK3
2        C1orf168
3             GBA
4           FCRL4
...           ...
10444         RS1
10445        UBA1
10446       PGAM4
10447     GPRASP1
10448       PNMA5

[10449 rows x 1 columns]


,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,...,FILTER,CONTEXT,src_vcf_id,tumor_bam_uuid,normal_bam_uuid,case_id,GDC_FILTER,COSMIC,MC3_Overlap,GDC_Validation_Status
0,BI,GRCh38,chr1,44008472,44008472,+,Silent,SNP,G,G,...,PASS,ACACCGGCGCA,85b191fa-e8f6-4d97-bc13-0bc236cee6c8,3d52eda9-5f36-4a55-8a3c-07a576a0ef1f,6c3037f4-9838-4b36-9551-99e91cb85ef0,14213209-2217-4812-9a19-d9b2b6718467,NaN,COSM6063461;COSM6063462,True,Valid
1,BI,GRCh38,chr1,44803280,44803280,+,Missense_Mutation,SNP,G,G,...,PASS,CCCCCGATCGA,85b191fa-e8f6-4d97-bc13-0bc236cee6c8,3d52eda9-5f36-4a55-8a3c-07a576a0ef1f,6c3037f4-9838-4b36-9551-99e91cb85ef0,14213209-2217-4812-9a19-d9b2b6718467,NaN,NaN,True,Valid
2,BI,GRCh38,chr1,56740711,56740711,+,Silent,SNP,C,C,...,PASS,TCTTTCGACTT,85b191fa-e8f6-4d97-bc13-0bc236cee6c8,3d52eda9-5f36-4a55-8a3c-07a576a0ef1f,6c3037f4-9838-4b36-9551-99e91cb85ef0,14213209-2217-4812-9a19-d9b2b6718467,common_in_exac,NaN,True,Unknown
3,BI,GRCh38,chr1,155233116,155233116,+,3'Flank,SNP,C,C,...,PASS,GCATGCAGCGG,85b191fa-e8f6-4d97-bc13-0bc236cee6c8,3d52eda9-5f36-4a55-8a3c-07a576a0ef1f,6c3037f4-9838-4b36-9551-99e91cb85ef0,14213209-2217-4812-9a19-d9b2b6718467,NaN,NaN,False,Unknown
4,BI,GRCh38,chr1,157581530,157581530,+,Splice_Site,SNP,C,C,...,PASS,ACAAACCTGAC,85b191fa-e8f6-4d97-bc13-0bc236cee6c8,3d52eda9-5f36-4a55-8a3c-07a576a0ef1f,6c3037f4-9838-4b36-9551-99e91cb85ef0,14213209-2217-4812-9a19-d9b2b6718467,NonExonic,NaN,True,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10444,BI,GRCh38,chrX,18642139,18642139,+,Silent,SNP,C,C,...,PASS,CGGTCCGAGTT,1d4f6015-1138-4d11-ae68-5034b9af7511,810c44b5-8901-4877-8ca6-bedfd040e0c2,e052d1b5-8959-43df-916a-8dce7977fdc0,200f77a5-803c-4891-980b-5c3b08c91254,NaN,NaN,True,Unknown
10445,BI,GRCh38,chrX,47201524,47201524,+,Missense_Mutation,SNP,T,T,...,PASS,CGGGTTTGAGA,1d4f6015-1138-4d11-ae68-5034b9af7511,810c44b5-8901-4877-8ca6-bedfd040e0c2,e052d1b5-8959-43df-916a-8dce7977fdc0,200f77a5-803c-4891-980b-5c3b08c91254,NaN,NaN,True,Unknown
10446,BI,GRCh38,chrX,77969434,77969434,+,Missense_Mutation,SNP,T,T,...,PASS,CACTGTCCAGA,1d4f6015-1138-4d11-ae68-5034b9af7511,810c44b5-8901-4877-8ca6-bedfd040e0c2,e052d1b5-8959-43df-916a-8dce7977fdc0,200f77a5-803c-4891-980b-5c3b08c91254,NaN,NaN,True,Unknown
10447,BI,GRCh38,chrX,102657224,102657224,+,Missense_Mutation,SNP,C,C,...,PASS,TCAGCCTGATC,1d4f6015-1138-4d11-ae68-5034b9af7511,810c44b5-8901-4877-8ca6-bedfd040e0c2,e052d1b5-8959-43df-916a-8dce7977fdc0,200f77a5-803c-4891-980b-5c3b08c91254,NaN,NaN,True,Unknown


54


In [43]:
#Handle features and genes_id#

#Retrieve Y (genes_id) and X (all other features)
Y = new_table.loc[:,["Hugo_Symbol"]].values
length = len(Y)

#Retrive features
X = new_table.drop(["Hugo_Symbol"], axis = 1)
thresh = length*0.90

In [44]:
#Handling NaN in cloumns below treshold 0.90(90%)#
#X  = X.dropna(axis=1,thresh=int(thresh))
#X.isna().sum().sort_values(ascending=False)
X.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Data columns (total 101 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    Entrez_Gene_Id               10449 non-null  int64  
 1    Center                       10449 non-null  object 
 2    NCBI_Build                   10449 non-null  object 
 3    Chromosome                   10449 non-null  object 
 4    Start_Position               10449 non-null  int64  
 5    End_Position                 10449 non-null  int64  
 6    Strand                       10449 non-null  object 
 7    Variant_Classification       10449 non-null  object 
 8    Variant_Type                 10449 non-null  object 
 9    Reference_Allele             10449 non-null  object 
 10   Tumor_Seq_Allele1            10449 non-null  object 
 11   Tumor_Seq_Allele2            10449 non-null  object 
 12   dbSNP_RS                     9507 non-null   object 
 13  

In [57]:
#Fill NaN values with interpolation method pad-by using existing values#
X = X.interpolate(method="pad")
X.info()
print(X.isna().any().sum())
m = X.loc[:,X.isna().any()]
m

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Columns: 101 entries, Entrez_Gene_Id to GDC_Validation_Status
dtypes: bool(1), float64(21), int64(10), object(69)
memory usage: 8.0+ MB
17


,DISTANCE,TREMBL,SIFT,PolyPhen,INTRON,GMAF,AFR_MAF,AMR_MAF,EAS_MAF,EUR_MAF,SAS_MAF,CLIN_SIG,SOMATIC,PUBMED,HGVS_OFFSET,PHENO,GDC_FILTER
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,deleterious(0.01),benign(0.262),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,deleterious(0.01),benign(0.262),NaN,0.0002,0.0000,0.0,0.0,0.0,0.001,NaN,NaN,NaN,NaN,NaN,common_in_exac
3,1336.0,A0A068F658,deleterious(0.01),benign(0.262),NaN,0.0002,0.0000,0.0,0.0,0.0,0.001,NaN,NaN,NaN,NaN,NaN,common_in_exac
4,1336.0,A0A068F658,deleterious(0.01),benign(0.262),7/11,0.0002,0.0000,0.0,0.0,0.0,0.001,NaN,NaN,NaN,NaN,NaN,NonExonic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10444,2205.0,A0A024R0J1,tolerated(0.32),benign(0.026),9/22,0.0042,0.0159,0.0,0.0,0.0,0.000,likely_pathogenic;pathogenic,1;0,25105660,1.0,1;0,common_in_exac;gdc_pon
10445,2205.0,A0A024R1A3,deleterious(0),probably_damaging(0.917),9/22,0.0042,0.0159,0.0,0.0,0.0,0.000,likely_pathogenic;pathogenic,1;0,25105660,1.0,1;0,common_in_exac;gdc_pon
10446,2205.0,A0A024R1A3,tolerated(0.41),benign(0.075),9/22,0.0042,0.0159,0.0,0.0,0.0,0.000,likely_pathogenic;pathogenic,1;0,25105660,1.0,1;0,common_in_exac;gdc_pon
10447,2205.0,A0A024R1A3,deleterious(0),possibly_damaging(0.754),9/22,0.0042,0.0159,0.0,0.0,0.0,0.000,likely_pathogenic;pathogenic,1;0,25105660,1.0,1;0,common_in_exac;gdc_pon


In [6]:
obj_df = X.select_dtypes(include=['object'])
X=pd.get_dummies(data=X,columns=obj_df.columns)
X.info()
X

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Columns: 119508 entries, Entrez_Gene_Id to GDC_Validation_Status_Valid
dtypes: bool(1), float64(1), int64(10), uint8(119496)
memory usage: 1.2 GB


,Entrez_Gene_Id,Start_Position,End_Position,t_depth,t_ref_count,t_alt_count,n_depth,ALLELE_NUM,TRANSCRIPT_STRAND,TSL,...,case_id_d9952e44-b573-4854-893e-b3bbc5287f2f,case_id_ddfdd05e-058c-4877-b389-724a6c101353,case_id_ebb834c6-374e-4077-a4ce-c74a850be358,case_id_ee6083aa-9453-44ff-8cc5-79b71fea289d,case_id_eec002c4-7602-4179-8eb3-2b0508d5ee1c,case_id_f2cc142b-9fc0-44d9-9ae3-558b4eace375,case_id_f98c183e-aaa1-40c4-83e7-c248cc3adb55,case_id_ffedc8be-1056-4205-b9d9-99b5bdb872db,GDC_Validation_Status_Unknown,GDC_Validation_Status_Valid
0,6536,44008472,44008472,103,65,38,132,1,-1,1.0,...,0,0,0,0,0,0,0,0,0,1
1,1263,44803280,44803280,84,51,33,112,1,1,1.0,...,0,0,0,0,0,0,0,0,0,1
2,199920,56740711,56740711,22,15,7,23,1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
3,2629,155233116,155233116,46,29,17,96,1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
4,83417,157581530,157581530,60,33,27,79,1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10444,6247,18642139,18642139,102,48,54,119,1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
10445,7317,47201524,47201524,43,32,11,47,1,1,1.0,...,0,0,0,0,0,0,0,0,1,0
10446,441531,77969434,77969434,205,112,93,216,1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
10447,9737,102657224,102657224,199,97,102,186,1,1,2.0,...,0,0,0,0,0,0,0,0,1,0


In [7]:
Y = LabelEncoder().fit_transform(Y.ravel())
Y

array([5426, 4497,  734, ..., 4374, 2387, 4526])

In [8]:
X = StandardScaler().fit_transform(X)
X

KeyboardInterrupt: 

In [ ]:
def forest_test(X, Y):
    X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, 
                                                        test_size = 0.20, 
                                                        random_state = 101)
    start = time.process_time()
    trainedforest = RandomForestClassifier(n_estimators=100).fit(X_Train,Y_Train)
    print(time.process_time() - start)
    predictionforest = trainedforest.predict(X_Test)
    print(confusion_matrix(Y_Test,predictionforest))
    print(classification_report(Y_Test,predictionforest))

In [ ]:
forest_test(X, Y)